In [1]:
from PIL import Image
import pytesseract
from pytesseract import Output
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os 

In [2]:
folders = [x[0] for x in os.walk("/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/Companies")]

In [6]:
folders = [ x for x in folders if "Customers" not in x ]
folders = [ x for x in folders if "Suppliers" not in x ]

In [8]:
len(folders)

272

In [22]:
tickers = pd.read_excel("/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/thao_tickers.xlsx")

In [23]:
tickers

,tickers
0,FRPT
1,VWR
2,SKYS
3,BSGM
4,ANN
...,...
266,AXSM
267,ARTL
268,APLS
269,BGNE


In [25]:
np.save('thao_tickers', tickers["tickers"])

## All the functions for OCR

In [28]:
def get_crop_dimensions(image):
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    lower_orange = np.array([0, 50, 50])
    upper_orange = np.array([20, 255, 255])
    
    mask = cv2.inRange(hsv_image, lower_orange, upper_orange)
    
    kernel = np.ones((5, 5), np.uint8)
    mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
    
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])
    
    return sorted_contours[0][1][0][1], sorted_contours[0][0][0][0], sorted_contours[-1][2][0][0]

In [29]:
def image_process(image):

    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    inverted_image = 255 - gray_image

    threshold_value = 200  # Threshold value (0-255)
    _, binary_image = cv2.threshold(inverted_image, threshold_value, 255, cv2.THRESH_BINARY)

    edges = cv2.Canny(binary_image, 50, 150)

    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=100, maxLineGap=10)
    
    return lines

In [30]:
def get_images(image1, filtered_array):
    company = image1[:, :filtered_array[0,0]]
    indus = image1[:, filtered_array[0,0]:filtered_array[1,0]]
    mkt = image1[:, filtered_array[1,0]:filtered_array[2,0]]
    rel = image1[:, filtered_array[2,0]:filtered_array[3,0]]
    per = image1[:, filtered_array[3,0]:filtered_array[4,0]]
    cat = image1[:, filtered_array[4,0]:filtered_array[5,0]]
    suprev = image1[:, filtered_array[5,0]:filtered_array[6,0]]
    source = image1[:, filtered_array[6,0]:filtered_array[7,0]]
    size = image1[:, filtered_array[7,0]:]
    
    return [company, indus, mkt, rel, per, cat, suprev, source, size]

In [130]:
def get_supplier_dict():
    dict1 = {'Company': [], 'Industry': [], 'Mkt Cap (M)':[],'Relationship Size (M)':[], 
         '% Cost':[], 'Cost Cat':[], 'Supplier Rev':[], 'Source':[], 'Size Source': []   
            }
    
    return dict1

In [131]:
def get_customer_dict():
    dict1 = {'Company': [], 'Industry': [], 'Mkt Cap (M)':[],'Relationship Size (M)':[], 
         '% Rev':[], '% Customer Cost':[], 'Customer Cost Cat':[], 'Source':[], 'Size Source': []   
            }

    return dict1

In [32]:
def process(image):
    imgss = cv2.resize(image, None, fx=1.3, fy=1.3, interpolation=cv2.INTER_CUBIC)
    text = pytesseract.image_to_string(imgss, lang='eng', config='--psm 11')
    return text

In [33]:
def text_clean(text):
    lines = text.split("\n")
    lines = [x for x in lines if x]
    return lines 

In [34]:
def find_rows(image):
    text = process(image)
    lines = text_clean(text)
    return len(lines)

In [35]:
def clean_lines(lines):
    big_lines = []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        if y1 - y2 >500:
            big_lines.append([x1, y1, x2, y2])
            
    return big_lines

In [36]:
def filtering_lines(big_lines):
    array = np.array(big_lines)
    sort_array = array[np.argsort(array[:, 0])]

    for row in sort_array:
        if row[3] > 0:
            row[3] = 0

    diff = np.abs(np.diff(sort_array[:, 0]))
    mask = np.concatenate(([True], diff > 5))

    return sort_array[mask]    

In [37]:
columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Cost',
           'Cost Cat', 'Supplier Rev', 'Source', 'Size Source']

### Loading the ticker file

In [121]:
tickers = np.load('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/thao_tickers.npy', allow_pickle=True)

## Main function

1. The current code is only running the code for pictures in Suppliers (folders[0])
2. You can change to customers or add an additional for loop for all the images. 

In [120]:
np.where(tickers=="MRUS")[0]

array([270])

In [146]:
# Suppliers
folders = ['Suppliers', 'Customers']
main_dir = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/Companies'
tickers = ['LILAK','AMSC','HRNNF','LCTX', 'LIN','KOPN','SNMSF','CAKE','BAMM','HOME','DLTH']
for ticker in tickers:
    folder_path = os.path.join(os.path.join(main_dir, ticker), folders[0])
    pics = os.listdir(folder_path)
    print('Number of images for ' + folders[0] + ' for ' + ticker + ' is ' + str(len(pics)))
    print('          ' + ticker + '          ')

    for pic in pics:
        image = cv2.imread(os.path.join(folder_path, pic))
        image = image[279:-75,:]
        y1, x1, x2 = get_crop_dimensions(image)
        image = image[y1:,x1-5:x2+20]

        print('Ticker: ' + ticker + ' ---  ' + 'Image number: ' + pic.split('.')[0])
        data = get_supplier_dict()
        lines = image_process(image)
        big_lines = clean_lines(lines)
        filtered_array = filtering_lines(big_lines)
        # check if filtered_array has the expected number of elements
        if len(filtered_array) < 8:
            filtered_array = np.array([[ 364,  652,  364,    0],
                                       [ 664,  652,  664,    0],
                                       [ 803,  652,  803,    0],
                                       [1105,  652, 1105,    0],
                                       [1177,  652, 1177,    0],
                                       [1271,  652, 1271,    0],
                                       [1450,  652, 1450,    0],
                                       [1713,  652, 1713,    0]])
#             print("Error: filtered_array has insufficient elements")
#             error_companies.append(ticker)

        images = get_images(image, filtered_array)
        rows = find_rows(images[0])

        print("Number of companies:", rows)

        for i in range(len(images)):
            text = process(images[i])
            lines = text_clean(text)
            if len(lines)==rows:
                print("Successful")
            else: 
                lines=['--'] * rows
                print("Changed/Editted")
            data[columns[i]] = lines

        df = pd.DataFrame(data)

        csv_path = os.path.join(os.path.join('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs',ticker), folders[0])
        csv_name = os.path.join(csv_path, pic).split('.')[0]+'.csv'

        df.to_csv(csv_name, sep=',', index =False)

    print('--------------------------------------------------------------------------------------')

Number of images for Suppliers for LILAK is 1
          LILAK          
Ticker: LILAK ---  Image number: S 1
Number of companies: 18
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
--------------------------------------------------------------------------------------
Number of images for Suppliers for AMSC is 1
          AMSC          
Ticker: AMSC ---  Image number: S 1
Number of companies: 8
Successful
Successful
Successful
Changed/Editted
Changed/Editted
Changed/Editted
Changed/Editted
Successful
Changed/Editted
--------------------------------------------------------------------------------------
Number of images for Suppliers for HRNNF is 1
          HRNNF          
Ticker: HRNNF ---  Image number: S 1
Number of companies: 8
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
Successful
--------------------------------------------------------------------------------------
Number of images for Su

In [140]:
np.where(tickers=="MCFT")[0]

array([171])

In [152]:
# Customers
customer_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Rev',
           '% Customer Cost', 'Customer Cost Cat', 'Source', 'Size Source']
folders = ['Suppliers', 'Customers']
main_dir = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/Companies'
error_companies = []
tickers = ['GNSS']
for ticker in tickers:
    folder_path = os.path.join(os.path.join(main_dir, ticker), folders[1])
    pics = os.listdir(folder_path)
    print('Number of images for ' + folders[1] + ' for ' + ticker + ' is ' + str(len(pics)))
    print('          ' + ticker + '          ')

    for pic in pics:
        image = cv2.imread(os.path.join(folder_path, pic))
        image = image[279:-75,:]
        y1, x1, x2 = get_crop_dimensions(image)
        image = image[y1:,x1-5:x2+20]

        print('Ticker: ' + ticker + ' ---  ' + 'Image number: ' + pic.split('.')[0])
        data = get_customer_dict()
        lines = image_process(image)
        big_lines = clean_lines(lines)
        filtered_array = filtering_lines(big_lines)
        
        # check if filtered_array has the expected number of elements
        if len(filtered_array) < 8:
            filtered_array = np.array([[ 364,  652,  364,    0],
                                       [ 664,  652,  664,    0],
                                       [ 803,  652,  803,    0],
                                       [1105,  652, 1105,    0],
                                       [1177,  652, 1177,    0],
                                       [1271,  652, 1271,    0],
                                       [1450,  652, 1450,    0],
                                       [1713,  652, 1713,    0]])
#             error_companies.append(ticker)
#             continue
        
        images = get_images(image, filtered_array)
        rows = find_rows(images[0])

        print("Number of companies:", rows)

        for i in range(len(images)):
            text = process(images[i])
            lines = text_clean(text)
            if len(lines)==rows:
                print("Successful")
            else: 
                lines=['--'] * rows
                print("Changed/Editted")
            data[customer_columns[i]] = lines

        df = pd.DataFrame(data)

        csv_path = os.path.join(os.path.join('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs',ticker), folders[1])
        csv_name = os.path.join(csv_path, pic).split('.')[0]+'.csv'

        df.to_csv(csv_name, sep=',', index =False)

    print('--------------------------------------------------------------------------------------')

print('Tickers with error: filtered_array has insufficient elements\n' + str(error_companies))

Number of images for Customers for GNSS is 3
          GNSS          
Ticker: GNSS ---  Image number: C 1
Number of companies: 21
Successful
Successful
Successful
Successful
Successful
Changed/Editted
Changed/Editted
Successful
Successful
Ticker: GNSS ---  Image number: C 2
Number of companies: 21
Successful
Successful
Successful
Changed/Editted
Changed/Editted
Changed/Editted
Changed/Editted
Successful
Changed/Editted
Ticker: GNSS ---  Image number: C 3


error: OpenCV(4.7.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [144]:
str(error_companies)

"['LIN', 'OLED', 'OLED', 'FCEL', 'SNMSF', 'VZIO', 'RHHBY', 'RHHBY', 'RHHBY', 'RHHBY', 'RHHBY', 'LFUS', 'GTLS', 'IOTS']"

In [159]:
# Customers
customer_columns = ['Company', 'Industry', 'Mkt Cap (M)', 'Relationship Size (M)', '% Rev',
           '% Customer Cost', 'Customer Cost Cat', 'Source', 'Size Source']
folders = ['Suppliers', 'Customers']
main_dir = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/Companies'
error_companies = []
tickers = ['GNSS']
for ticker in tickers:

    print('Number of images for ' + folders[1] + ' for ' + ticker + ' is ' + str(len(pics)))
    print('          ' + ticker + '          ')

    image = cv2.imread('/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/Companies/GNSS/Customers/C 3.png')
    image = image[410:-75,:]
    y1, x1, x2 = get_crop_dimensions(image)
    image = image[y1:,x1-5:x2+20]

    print('Ticker: ' + ticker + ' ---  ' + 'Image number: ' + pic.split('.')[0])
    data = get_customer_dict()
    lines = image_process(image)
    big_lines = clean_lines(lines)
    filtered_array = filtering_lines(big_lines)

    # check if filtered_array has the expected number of elements
    if len(filtered_array) < 8:
        filtered_array = np.array([[ 364,  652,  364,    0],
                                   [ 664,  652,  664,    0],
                                   [ 803,  652,  803,    0],
                                   [1105,  652, 1105,    0],
                                   [1177,  652, 1177,    0],
                                   [1271,  652, 1271,    0],
                                   [1450,  652, 1450,    0],
                                   [1713,  652, 1713,    0]])
#             error_companies.append(ticker)
#             continue

    images = get_images(image, filtered_array)
    rows = find_rows(images[0])

    print("Number of companies:", rows)

    for i in range(len(images)):
        text = process(images[i])
        lines = text_clean(text)
        if len(lines)==rows:
            print("Successful")
        else: 
            lines=['--'] * rows
            print("Changed/Editted")
        data[customer_columns[i]] = lines

    df = pd.DataFrame(data)

    csv_path = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers'
    csv_name = '/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers/C 3.csv'

    df.to_csv(csv_name, sep=',', index =False)

    print('--------------------------------------------------------------------------------------')

print('Tickers with error: filtered_array has insufficient elements\n' + str(error_companies))

Number of images for Customers for GNSS is 3
          GNSS          


IndexError: list index out of range

In [153]:
csv_path

'/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers'

In [154]:
csv_name

'/Users/minhthao/Desktop/MSDA/Summer 2023/Bloomberg/SPLC Images/CSVs/GNSS/Customers/C 2.csv'